# COGS 108 - Assignment 3: Data Privacy

## Important Reminders

- Rename this file to 'A3_$####.ipynb', replacing with your unique ID (first letter of your last name, followed by the last 4 digits of your student ID number), before you submit it. Submit it to TritonED.
- Do not change / update / delete any existing cells with 'assert' in them. These are the tests used to check your assignment. 
    - Changing these will be flagged for attempted cheating. 
- This assignment has hidden tests: tests that are not visible here, but that will be run on your submitted file. 
    - This means passing all the tests you can see in the notebook here does not guarantee you have the right answer!

## Overview

We have discussed in lecture the importance and the mechanics of protecting individuals privacy when they are included in datasets. 

One method to do so is the Safe Harbor Method. The Safe Harbour method specifies how to protect individual's identities by telling us which tells us which information to remove from a dataset in order to avoid accidently disclosing personal information. 

In this assignment, we will explore web scraping, which can often include personally identifiable information, how identity can be decoded from badly anonymized datasets, and also explore using Safe Harbour to anonymize datasets properly. 

The topics covered in this assignment are mainly covered in the 'DataGathering' and 'DataPrivacy&Anonymization' Tutorial notebooks.

### Installing new packages

In the first part of the assignment we will understand how we can scrape the web for data. You have to use the Beautiful Soup library in Python for scraping the data. 

The library is not installed in Anaconda version, therefore to install a new library for Anaconda, we can use the conda package manager, which the cell below does for you. 

In [2]:
# Run this cell to install beautifulsoup4
#  You only need to do the installation once
#    Once you have run it you can comment these two lines so that the cell doesn't execute everytime.
import sys
!conda install --yes --prefix {sys.prefix} beautifulsoup4

Solving environment: done

# All requested packages already installed.



### Imports

In [4]:
# Imports - these provided for you. Do not import any other packages
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup

## Part 1: Web Scraping 

### Scraping Rules

1) If you are using another organizations website for scraping, make sure to check the website's terms & conditions. 

2) Do not request data from the website too aggressively (quickly) with your program (also known as spamming), as this may break the website. Make sure your program behaves in a reasonable manner (i.e. acts like a human). One request for one webpage per second is good practice.

3) The layout of a website may change from time to time. Because of this, if you're scraping website, make sure to revisit the site and rewrite your code as needed.

In [ ]:
# This cell will help you understand the permission issues related to accessing a page
# Uncomment the two lines, run them, see what error you get, comment them again

#page_source = requests.get('http://www.aflcio.org/Legislation-and-Politics/Legislative-Alerts')
#page_soup = BeautifulSoup(page_source.content, 'html.parser')

In [6]:
page_source = requests.get('http://www.aflcio.org/Legislation-and-Politics/Legislative-Alerts')
page_soup = BeautifulSoup(page_source.content, 'html.parser')

#### What is the error that you got, and why did you get it?

YOUR ANSWER HERE

In [8]:
# 1a) Web Scrape
# We will first retrieve the contents on a page and examine them a bit.

# Make a variable called 'wiki', that stores the following URL (as a string):
#  'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_population'
# To open the URL you can use 'requests.get' as shown in the cell above. Call this variable 'page'
# After that use BeautifulSoup Library to open the URL and assign it to an object called 'soup'

# YOUR CODE HERE
wiki = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_population'
page = requests.get(wiki)
soup = BeautifulSoup(page.content, 'html.parser')

In [9]:
assert wiki
assert page
assert soup


In [12]:
# 1b) Checking Scrape Contents

# Extract the title from the page and save it in a variable called 'title_page'. 
#  Make sure you extract it as a string.
# To do so, you have to use the soup object created in the above cell. 
#  Hint: from your soup variable, you can access this with '.title.string'
# Make sure you print out and check the contents of 'title_page'. 
#  Note that it should not have any tags (such as '<title>' included in it).

title_page = soup.title.string
print(title_page)
assert (title_page == 'List of U.S. states and territories by population - Wikipedia')

List of U.S. states and territories by population - Wikipedia


In [ ]:
assert title_page
assert isinstance(title_page, str)


In [13]:
# 1c) Extracting Tables

# In order to extract the data we want, we'll start with extracting a data table of interest. 
#  Note that you can see this table by going to look at the link we scraped.
# Use the soup object and call a method called find, which will and extract the first table in scraped webpage. 
#  Note: you need to search for the name 'table', and set the 'class_' argument as 'wikitable sortable'.

# YOUR CODE HERE
right_table = soup.find('table', class_='wikitable sortable')
#print(right_table)

<table class="wikitable sortable" style="width:100%; text-align:center;">
<tr style="vertical-align: top;">
<th style="vertical-align: middle">Rank in the <a href="/wiki/List_of_states_and_territories_of_the_United_States#States" title="List of states and territories of the United States">fifty states</a>, 2017</th>
<th style="vertical-align: middle">Rank in all states &amp; territories, 2010</th>
<th style="width: 20%; vertical-align: middle"><b>State or territory</b></th>
<th style="width: 20%; vertical-align: middle"><b>Equivalent country</b></th>
<th style="vertical-align: middle">Population estimate, July 1, 2017<sup class="reference" id="cite_ref-4"><a href="#cite_note-4">[4]</a></sup></th>
<th style="vertical-align: middle">Census population, April 1, 2010</th>
<th style="vertical-align: middle">Total seats in House of Representatives, 2013–2023</th>
<th style="vertical-align: middle">Estimated pop. per House seat, 2017</th>
<th style="vertical-align: middle">Census pop. per Hou

In [14]:
assert right_table
assert isinstance(right_table, bs4.element.Tag)
assert right_table.name == 'table'

In [15]:
# Extract the data from the table into lists.
#  Note: This code provided for you. Do read through it and try to see how it works.

lst_a, lst_b, lst_c = [], [], []

for row in right_table.findAll('tr'):
    
    cells = row.findAll('td')
    
    # Skips rows that aren't 10 columns long (like the heading)
    if len(cells) != 10:
        continue

    # This catches when the name cells stops having a link
    #  and ends, skipping the last (summary rows)
    try:
        lst_a.append(cells[2].find('a').text)
        lst_b.append(cells[4].find(text=True))
        lst_c.append(cells[5].find(text=True))
    except:
        break

In [45]:
# 1d) Collecting into a dataframe

# Create a dataframe 'my_df' and add the data from the lists above to it. 
#  'lst_a' is the state or territory name. Set the column name as 'State', and make this the index
#  'lst_b' is the population estimate. Add it to the dataframe, and set the column name as 'Population Estimate'
#  'lst_c' is the census population. Add it to the dataframe, and set the column name as 'Census Population'

my_df = pd.DataFrame({'State':lst_a, 'Population Estimate': lst_b, 'Census Population': lst_c})
my_df = my_df[['State', 'Population Estimate', 'Census Population']]
my_df = my_df.set_index('State')

print(my_df.head())

             Population Estimate Census Population
State                                             
California            39,536,653        37,252,895
Texas                 28,304,596        25,146,105
Florida               20,984,400        18,804,623
New York              19,849,399        19,378,087
Pennsylvania          12,805,537        12,702,887


In [36]:
assert isinstance (my_df, pd.DataFrame)
assert my_df.index.name == 'State'
assert list(my_df.columns) == ['Population Estimate', 'Census Population']


In [46]:
# 1e) Using the data
# What is the Population Estimate of California? Save this answer to a variable called 'calif_pop'
# Notes:
#  Extract this value programmatically from your dataframe (as in, don't set it explicitly, as 'cf = 123')
#    You can use '.loc' to extract a particular value from a dataframe.
#  The data in your dataframe will be strings - that's fine, leave them as strings (don't typecast).

calif_pop = my_df.loc['California', 'Population Estimate']
print(calif_pop)

39,536,653


In [47]:
assert calif_pop


## Part 2: Identifying Data

Data Files:
- anon_user_dat.json
- employee_info.json

You will first be working with a file called 'anon_user_dat.json'. This file that contains information about some (fake) Tinder users. When creating an account, each Tinder user was asked to provide their first name, last name, work email (to verify the disclosed workplace), age, gender, phone # and zip code. Before releasing this data, a data scientist cleaned the data to protect the privacy of Tinder's users by removing the obvious personal identifiers: phone #, zip code, and IP address. However, the data scientist chose to keep each users' email addresses because when they visually skimmed a couple of the email addresses none of them seemed to have any of the user's actual names in them. This is where the data scientist made a huge mistake!

We will take advantage of having the work email addresses by finding the employee information of different companies and matching that employee information with the information we have, in order to identify the names of the secret Tinder users!

In [51]:
# 2a) Load in the 'cleaned' data 

# Load the json file into a pandas dataframe. Call it 'df_personal'.

df_personal = pd.read_json('anon_user_dat.json')

In [53]:
print(df_personal.head())

   age                        email  gender
0   60  gshoreson0@seattletimes.com    Male
1   47           eweaben1@salon.com  Female
2   27      akillerby2@gravatar.com    Male
3   46            gsainz3@zdnet.com    Male
4   72     bdanilewicz4@4shared.com    Male


In [52]:
assert isinstance(df_personal, pd.DataFrame)


In [58]:
# 2b) Check the first 10 emails 

# Save the first 10 emails to a Series, and call it 'sample_emails'. 
# You should then and print out this Series. 
# The purpose of this is to get a sense of how these work emails are structured
#   and how we could possibly extract where each anonymous user seems to work.

sample_emails = df_personal['email'][:10]
print(sample_emails)

0    gshoreson0@seattletimes.com
1             eweaben1@salon.com
2        akillerby2@gravatar.com
3              gsainz3@zdnet.com
4       bdanilewicz4@4shared.com
5      sdeerness5@wikispaces.com
6         jstillwell6@ustream.tv
7         mpriestland7@opera.com
8       nerickssen8@hatena.ne.jp
9             hparsell9@xing.com
Name: email, dtype: object


In [59]:
assert isinstance(sample_emails, pd.Series)


In [61]:
# 2c) Extract the Company Name From the Email 

# Create a function with the following specifications:
#   Function Name: extract_company
#   Purpose: to extract the company of the email 
#          (i.e., everything after the @ sign but before the .)
#   Parameter(s): email (string)
#   Returns: The extracted part of the email (string)
#   Hint: This should take 1 line of code. Look into the find('') method. 
#
# You can start with this outline:
#   def extract_company(email):
#      return 
#
# Example Usage: 
#   extract_company("larhe@uber.com") should return "uber"
#   extract_company(“ds@cogs.edu”) should return “cogs”


def extract_company(email):
    return email[email.find('@') + 1 : email.find('.', email.find('@'))]

print(extract_company("gshoreson0@seattletimes.com"))

seattletimes


In [62]:
assert extract_company("gshoreson0@seattletimes.com") == "seattletimes"


With a little bit of basic sleuthing (aka googling) and web-scraping (aka selectively reading in html code) it turns out that you've been able to collect information about all the present employees/interns of the companies you are interested in. Specifically, on each company website, you have found the name, gender, and age of its employees. You have saved that info in employee_info.json and plan to see if, using this new information, you can match the Tinder accounts to actual names.

In [63]:
# 2d) Load in employee data 

# Load the json file into a pandas dataframe. Call it 'df_employee'.

df_employee = pd.read_json('employee_info.json')

In [64]:
assert isinstance(df_employee, pd.DataFrame)


In [70]:
# 2e) Match the employee name with company, age, gender 

# Create a function with the following specifications:
#   Function name: employee_matcher
#   Purpose: to match the employee name with the provided company, age, and gender
#   Parameter(s): company (string), age (int), gender (string)
#   Returns: The employee first_name and last_name like this: return first_name, last_name 
#   Note: If there are multiple employees that fit the same description, first_name and 
#         last_name should return a list of all possible first names and last name
#         i.e., ['Desmund', 'Kelby'], ['Shepley', 'Tichner']
#
# Hint:
# There are many different ways to code this.
# 1) An unelegant solution is to loop through df_employee 
#    and for each data item see if the company, age, and gender match
#    i.e., for i in range(0, len(df_employee)):
#              if (company == df_employee.ix[i,'company']):
#
# However! The solution above is very inefficient and long, 
# so you should try to look into this:
# 2) Google the df.loc method: It extracts pieces of the dataframe
#    if it fulfills a certain condition.
#    i.e., df_employee.loc[df_employee['company'] == company]
#    If you need to convert your pandas data series into a list,
#    you can do list(result) where result is a pandas "series"
# 
# You can start with this outline:
#   def employee_matcher(company, age, gender):
#      return first_name, last_name

def employee_matcher(company, age, gender):
    records = df_employee.loc[(df_employee.company == company) & (df_employee.age == age) & (df_employee.gender == gender)]
    return (list(records.first_name), list(records.last_name))

print(employee_matcher("google", 41, "Male"))

(['Maxwell'], ['Jorio'])


In [71]:
assert employee_matcher("google", 41, "Male") == (['Maxwell'], ['Jorio'])
assert employee_matcher("salon", 47, "Female") == (['Elenore'], ['Gravett'])


In [84]:
# 2f) Extract all the private data 

# - Create 2 empty lists called 'first_names' and 'last_names'
# - Loop through all the people we are trying to identify in df_personal
# - Call the extract_company function (i.e., extract_company(df_personal.ix[i, 'email']) )
# - Call the employee_matcher function 
# - Append the results of employee_matcher to the appropriate lists (first_names and last_names)

first_names = []
last_names = []

for i in range(len(df_personal)):
    company = extract_company(df_personal.loc[i, 'email'])
    record = employee_matcher(company, df_personal.loc[i, 'age'], df_personal.loc[i, 'gender'])
    first_names.append(record[0])
    last_names.append(record[1])

In [85]:
assert first_names[45:50]== [['Justino'], ['Tadio'], ['Kennith'], ['Cedric'], ['Amargo']]
assert last_names[45:50] == [['Corro'], ['Blackford'], ['Milton'], ['Yggo'], ['Grigor']]


In [86]:
# 2g) Add the names to the original 'secure' dataset! 

# We have done this last step for you below, all you need to do is run this cell.
# For your own personal enjoyment, you should also print out
#   the new df_personal with the identified people. 

df_personal['first_name'] = first_names
df_personal['last_name'] = last_names

In [87]:
print(df_personal.head())

   age                        email  gender first_name    last_name
0   60  gshoreson0@seattletimes.com    Male   [Gordon]  [DelaField]
1   47           eweaben1@salon.com  Female  [Elenore]    [Gravett]
2   27      akillerby2@gravatar.com    Male     [Abbe]  [Stockdale]
3   46            gsainz3@zdnet.com    Male    [Guido]    [Comfort]
4   72     bdanilewicz4@4shared.com    Male    [Brody]   [Pinckard]


We have now just discovered the 'anonymous' identities of all the registered Tinder users...awkward.

## Part 3: Anonymize Data

You are hopefully now convinced that with some seemingly harmless data a hacker can pretty easily discover the identities of certain users. Thus, we will now clean the original Tinder data ourselves according to the Safe Harbor Method in order to make sure that it has been *properly* cleaned...

In [90]:
# 3a) Load in personal data 

# Load the user_dat.json file into a pandas dataframe. Call it 'df_users'.
# Note: You might find that using the same method as A2 (or above) leads to an error.
# The file has a slightly different organization. 
#   Try googling the error and finding the fix for it.
# Hint: you can still use 'pd.read_json', you just need to add another argument.

# YOUR CODE HERE
df_users = pd.read_json('user_dat.json', lines=True)

In [91]:
assert isinstance(df_users, pd.DataFrame)


In [92]:
print(df_users.head())

   age                        email first_name  gender      ip_address  \
0   60  gshoreson0@seattletimes.com     Gordon    Male   230.97.219.70   
1   47           eweaben1@salon.com    Elenore  Female  202.253.80.173   
2   27      akillerby2@gravatar.com       Abbe    Male   15.120.128.79   
3   46            gsainz3@zdnet.com      Guido    Male  71.234.147.178   
4   72     bdanilewicz4@4shared.com      Brody    Male  68.192.188.136   

   last_name          phone    zip  
0  DelaField  (245)663-9899   6705  
1    Gravett  (445)749-6402  40330  
2  Stockdale  (312)997-8947  44139  
3    Comfort  (530)815-7417  44201  
4   Pinckard  (214)138-6981  72956  


In [100]:
# 3b) Drop personal attributes 

# Remove any personal information, following the Safe Harbour method.
# Based on the Safe Harbour method, remove any columns from df_users that contain personal information.
#   Note that details on the Safe Harbour method are covered in the Tutorials.

# Names
# Geographic Subdivisions smaller than a state**
# Dates (such as birth dates, etc), and all ages above 90
# Telephone Numbers
# Vehicle Identification Numbers
# Fax numbers
# Device identifiers and serial numbers
# Email addresses
# Web Universal Resource Locators (URLs)
# Social security numbers
# Internet Protocol (IP) addresses
# Medical record numbers
# Biometric identifiers, including finger and voice prints
# Health plan beneficiary numbers
# Full-face photographs and any comparable images
# Account numbers
# Certificate/license numbers
# Any other unique identifying number, characteristic, or code


df_users = df_users.drop(['first_name', 'last_name', 'email', 'ip_address', 'phone'], axis=1)


In [101]:
assert len(df_users.columns) == 3


In [102]:
# 3c) Drop ages that are above 90 

# Safe Harbour rule C:
#   Drop all the rows which have age greater than 90 from df_users

df_users = df_users.loc[df_users['age'] <= 90]

In [103]:
assert df_users.shape == (993, 3)


In [115]:
# 3d) Load in zip code data 

# Load the zip_pop.csv file into a (different) pandas dataframe. Call it 'df_zip'.
# Note that the zip data should be read in as strings, not ints, as would be the default. 
# In read_csv, use the parameter 'dtype' to specify to read 'zip' as str, and 'population' as int.

df_zip = pd.read_csv('zip_pop.csv', dtype={'zip':str})

zip           object
population     int64
dtype: object
     zip  population
0  01001       16769
1  01002       29049
2  01003       10372
3  01005        5079
4  01007       14649
<class 'numpy.int64'>


In [132]:
print(df_zip[:20])

      zip  population
0   01001       16769
1   01002       29049
2   01003       10372
3   01005        5079
4   01007       14649
5   01008        1263
6   01009         741
7   01010        3609
8   01011        1370
9   01012         661
10  01013       23188
11  01020       29668
12  01022        2451
13  01026         946
14  01027       17660
15  01028       15720
16  01029         789
17  01030       11669
18  01031        1308
19  01032         570


In [105]:
assert isinstance(df_zip, pd.DataFrame)


In [134]:
# 3e) Sort zipcodes into "Geographic Subdivision" 

# The Safe Harbour Method applies to "Geographic Subdivisions"
#   as opposed to each zipcode itself. 
# Geographic Subdivision:
#   All areas which share the first 3 digits of a zip code
#
# Count the total population for each geographic subdivision
# Warning: you have to be savy with a dictionary here
# To understand how a dictionary works, check the section materials,
#   use google and go to discussion sections!
#
# Instructions: 
# - Create an empty dictionary: zip_dict = {}
# - Loop through all the zip_codes in df_zip
# - Create a dictionary key for the first 3 digits of a zip_code in zip_dict
# - Continually add population counts to the key that contains the 
#     same first 3 digits of the zip code
#
# To extract the population you will find this code useful:
#   population = list(df_zip.loc[df_zip['zip'] == zip_code]['population'])
# To extract the first 3 digits of a zip_code you will find this code useful:
#   int(str(zip_code)[:3])
#
# Note: this code may take some time (many seconds, up to a minute or two) to run

zip_dict = {}
for ix, row in df_zip.iterrows():
    print(f'key: {int(str(row.zip[:3]))} | value: {row.population}')
    #zip_dict[int(row['zip'][:4])] = zip_dict.get(int(row['zip'][:4]), 0) + row.population
    

key: 10 | value: 16769
key: 10 | value: 29049
key: 10 | value: 10372
key: 10 | value: 5079
key: 10 | value: 14649
key: 10 | value: 1263
key: 10 | value: 741
key: 10 | value: 3609
key: 10 | value: 1370
key: 10 | value: 661
key: 10 | value: 23188
key: 10 | value: 29668
key: 10 | value: 2451
key: 10 | value: 946
key: 10 | value: 17660
key: 10 | value: 15720
key: 10 | value: 789
key: 10 | value: 11669
key: 10 | value: 1308
key: 10 | value: 570
key: 10 | value: 6227
key: 10 | value: 2021
key: 10 | value: 5250
key: 10 | value: 5109
key: 10 | value: 838
key: 10 | value: 2545
key: 10 | value: 1336
key: 10 | value: 39880
key: 10 | value: 2530
key: 10 | value: 1685
key: 10 | value: 1851
key: 10 | value: 21103
key: 10 | value: 8534
key: 10 | value: 15284
key: 10 | value: 11150
key: 10 | value: 430
key: 10 | value: 64
key: 10 | value: 1902
key: 10 | value: 8469
key: 10 | value: 629
key: 10 | value: 1596
key: 10 | value: 1478
key: 10 | value: 5792
key: 10 | value: 319
key: 10 | value: 17527
key: 10

key: 28 | value: 7831
key: 28 | value: 7496
key: 28 | value: 278
key: 28 | value: 32611
key: 28 | value: 6135
key: 28 | value: 18082
key: 28 | value: 6111
key: 28 | value: 5377
key: 28 | value: 791
key: 28 | value: 2172
key: 28 | value: 7704
key: 28 | value: 5976
key: 28 | value: 3515
key: 28 | value: 4608
key: 28 | value: 784
key: 28 | value: 5405
key: 28 | value: 136
key: 28 | value: 3492
key: 28 | value: 3400
key: 28 | value: 1902
key: 28 | value: 23335
key: 28 | value: 1392
key: 28 | value: 16095
key: 28 | value: 22551
key: 28 | value: 8412
key: 28 | value: 570
key: 28 | value: 6738
key: 28 | value: 45199
key: 28 | value: 14
key: 28 | value: 26164
key: 28 | value: 19385
key: 28 | value: 33396
key: 28 | value: 16951
key: 28 | value: 17171
key: 28 | value: 218
key: 28 | value: 235
key: 28 | value: 5860
key: 28 | value: 307
key: 28 | value: 430
key: 28 | value: 15780
key: 28 | value: 20371
key: 28 | value: 7841
key: 28 | value: 14184
key: 28 | value: 10627
key: 28 | value: 29269
key: 

key: 54 | value: 4397
key: 54 | value: 14449
key: 54 | value: 0
key: 54 | value: 7198
key: 54 | value: 1303
key: 54 | value: 93
key: 54 | value: 1654
key: 54 | value: 1498
key: 54 | value: 7690
key: 54 | value: 3174
key: 54 | value: 5909
key: 54 | value: 682
key: 54 | value: 1767
key: 54 | value: 9341
key: 56 | value: 11916
key: 56 | value: 161
key: 56 | value: 17169
key: 56 | value: 1141
key: 56 | value: 423
key: 56 | value: 574
key: 56 | value: 1123
key: 56 | value: 1546
key: 56 | value: 776
key: 56 | value: 547
key: 56 | value: 970
key: 56 | value: 2868
key: 56 | value: 3508
key: 56 | value: 56
key: 56 | value: 1402
key: 56 | value: 1660
key: 56 | value: 5620
key: 56 | value: 6773
key: 56 | value: 80
key: 56 | value: 2166
key: 56 | value: 634
key: 56 | value: 4314
key: 56 | value: 2574
key: 56 | value: 1705
key: 56 | value: 1064
key: 56 | value: 4984
key: 56 | value: 2232
key: 56 | value: 163
key: 56 | value: 3296
key: 56 | value: 2168
key: 56 | value: 458
key: 56 | value: 1251
key:

key: 88 | value: 1106
key: 88 | value: 10063
key: 88 | value: 17
key: 89 | value: 55223
key: 89 | value: 41153
key: 89 | value: 13982
key: 100 | value: 21102
key: 100 | value: 81410
key: 100 | value: 56024
key: 100 | value: 3089
key: 100 | value: 7135
key: 100 | value: 3011
key: 100 | value: 6988
key: 100 | value: 61347
key: 100 | value: 31834
key: 100 | value: 50984
key: 100 | value: 24090
key: 100 | value: 27700
key: 100 | value: 31959
key: 100 | value: 54183
key: 100 | value: 16575
key: 100 | value: 5229
key: 100 | value: 42870
key: 100 | value: 0
key: 100 | value: 43631
key: 100 | value: 31924
key: 100 | value: 60998
key: 100 | value: 59283
key: 100 | value: 94600
key: 100 | value: 34003
key: 100 | value: 59707
key: 100 | value: 45141
key: 100 | value: 76003
key: 100 | value: 26999
key: 100 | value: 56438
key: 100 | value: 57331
key: 100 | value: 53926
key: 100 | value: 38908
key: 100 | value: 33969
key: 100 | value: 24711
key: 100 | value: 17416
key: 100 | value: 20300
key: 100 | 

key: 130 | value: 3631
key: 130 | value: 1169
key: 130 | value: 4871
key: 130 | value: 1809
key: 130 | value: 4409
key: 130 | value: 168
key: 130 | value: 22156
key: 130 | value: 30124
key: 130 | value: 2606
key: 131 | value: 2508
key: 131 | value: 85
key: 131 | value: 326
key: 131 | value: 15567
key: 131 | value: 6175
key: 131 | value: 2338
key: 131 | value: 1700
key: 131 | value: 1973
key: 131 | value: 287
key: 131 | value: 6356
key: 131 | value: 224
key: 131 | value: 3457
key: 131 | value: 188
key: 131 | value: 6382
key: 131 | value: 2328
key: 131 | value: 1174
key: 131 | value: 314
key: 131 | value: 124
key: 131 | value: 37205
key: 131 | value: 3895
key: 131 | value: 4244
key: 131 | value: 119
key: 131 | value: 6617
key: 131 | value: 534
key: 131 | value: 61
key: 131 | value: 4488
key: 131 | value: 596
key: 131 | value: 6451
key: 131 | value: 2873
key: 131 | value: 1531
key: 131 | value: 1926
key: 131 | value: 2443
key: 131 | value: 1281
key: 131 | value: 10750
key: 131 | value: 79

...

key: 836 | value: 153
key: 836 | value: 206
key: 836 | value: 5903
key: 836 | value: 3300
key: 836 | value: 3438
key: 836 | value: 37008
key: 836 | value: 129
key: 836 | value: 9976
key: 836 | value: 636
key: 836 | value: 47098
key: 836 | value: 20789
key: 836 | value: 3238
key: 836 | value: 552
key: 836 | value: 28611
key: 836 | value: 1672
key: 836 | value: 4397
key: 836 | value: 582
key: 836 | value: 174
key: 836 | value: 5890
key: 836 | value: 9881
key: 836 | value: 52
key: 836 | value: 7365
key: 836 | value: 553
key: 836 | value: 33
key: 836 | value: 8583
key: 836 | value: 4347
key: 836 | value: 44
key: 836 | value: 46224
key: 836 | value: 29697
key: 837 | value: 22293
key: 837 | value: 16565
key: 837 | value: 39628
key: 837 | value: 26631
key: 837 | value: 29732
key: 837 | value: 50034
key: 837 | value: 8148
key: 837 | value: 28103
key: 837 | value: 20290
key: 837 | value: 14523
key: 838 | value: 6150
key: 838 | value: 68
key: 838 | value: 744
key: 838 | value: 1254
key: 838 | va

key: 875 | value: 2747
key: 875 | value: 199
key: 875 | value: 1007
key: 875 | value: 1345
key: 875 | value: 311
key: 875 | value: 130
key: 875 | value: 356
key: 875 | value: 91
key: 875 | value: 460
key: 875 | value: 1419
key: 875 | value: 1003
key: 875 | value: 3384
key: 875 | value: 405
key: 875 | value: 260
key: 875 | value: 72
key: 875 | value: 880
key: 875 | value: 3417
key: 875 | value: 3600
key: 875 | value: 974
key: 875 | value: 368
key: 875 | value: 18223
key: 875 | value: 185
key: 875 | value: 1270
key: 875 | value: 2957
key: 875 | value: 493
key: 875 | value: 269
key: 875 | value: 923
key: 875 | value: 247
key: 875 | value: 17950
key: 875 | value: 928
key: 875 | value: 1118
key: 875 | value: 305
key: 875 | value: 3325
key: 875 | value: 1209
key: 875 | value: 61
key: 875 | value: 2417
key: 875 | value: 6359
key: 875 | value: 606
key: 875 | value: 1445
key: 875 | value: 576
key: 875 | value: 332
key: 875 | value: 478
key: 875 | value: 3000
key: 875 | value: 1634
key: 875 | va

key: 923 | value: 5268
key: 923 | value: 36
key: 923 | value: 313
key: 923 | value: 2270
key: 923 | value: 156
key: 923 | value: 2534
key: 923 | value: 54858
key: 923 | value: 33237
key: 923 | value: 42400
key: 923 | value: 4894
key: 923 | value: 1379
key: 923 | value: 52606
key: 924 | value: 1932
key: 924 | value: 58271
key: 924 | value: 28873
key: 924 | value: 56689
key: 924 | value: 15271
key: 924 | value: 49410
key: 924 | value: 26214
key: 925 | value: 20970
key: 925 | value: 84519
key: 925 | value: 53778
key: 925 | value: 47672
key: 925 | value: 44001
key: 925 | value: 58002
key: 925 | value: 35000
key: 925 | value: 75196
key: 925 | value: 1162
key: 925 | value: 50216
key: 925 | value: 18644
key: 925 | value: 3810
key: 925 | value: 4734
key: 925 | value: 33555
key: 925 | value: 44734
key: 925 | value: 39457
key: 925 | value: 6643
key: 925 | value: 3926
key: 925 | value: 30815
key: 925 | value: 73722
key: 925 | value: 39076
key: 925 | value: 50320
key: 925 | value: 1661
key: 925 | 

key: 954 | value: 950
key: 954 | value: 401
key: 954 | value: 1592
key: 954 | value: 297
key: 954 | value: 374
key: 954 | value: 1592
key: 954 | value: 15585
key: 954 | value: 3747
key: 954 | value: 10841
key: 954 | value: 2147
key: 954 | value: 459
key: 954 | value: 2516
key: 954 | value: 70
key: 954 | value: 44
key: 954 | value: 631
key: 954 | value: 426
key: 954 | value: 82
key: 954 | value: 5418
key: 954 | value: 14799
key: 954 | value: 675
key: 954 | value: 2014
key: 954 | value: 3178
key: 954 | value: 233
key: 954 | value: 748
key: 954 | value: 2093
key: 954 | value: 4750
key: 954 | value: 16852
key: 954 | value: 1507
key: 954 | value: 387
key: 954 | value: 11213
key: 954 | value: 1393
key: 954 | value: 11256
key: 954 | value: 2556
key: 954 | value: 795
key: 954 | value: 2893
key: 954 | value: 3104
key: 954 | value: 504
key: 954 | value: 2285
key: 954 | value: 3413
key: 954 | value: 1417
key: 954 | value: 169
key: 954 | value: 2673
key: 954 | value: 2083
key: 954 | value: 946
key

key: 976 | value: 780
key: 976 | value: 5096
key: 976 | value: 1466
key: 976 | value: 1361
key: 976 | value: 212
key: 976 | value: 165
key: 976 | value: 124
key: 976 | value: 387
key: 976 | value: 76
key: 976 | value: 507
key: 976 | value: 570
key: 976 | value: 99
key: 976 | value: 1313
key: 977 | value: 58993
key: 977 | value: 40583
key: 977 | value: 5814
key: 977 | value: 120
key: 977 | value: 61
key: 977 | value: 38
key: 977 | value: 4530
key: 977 | value: 271
key: 977 | value: 91
key: 977 | value: 252
key: 977 | value: 278
key: 977 | value: 139
key: 977 | value: 770
key: 977 | value: 2786
key: 977 | value: 165
key: 977 | value: 73
key: 977 | value: 496
key: 977 | value: 1941
key: 977 | value: 11018
key: 977 | value: 11912
key: 977 | value: 386
key: 977 | value: 129
key: 977 | value: 44
key: 977 | value: 2016
key: 977 | value: 18574
key: 977 | value: 33554
key: 977 | value: 101
key: 977 | value: 6294
key: 977 | value: 6757
key: 977 | value: 3987
key: 978 | value: 21521
key: 978 | va

In [124]:
assert isinstance(zip_dict, dict)
assert zip_dict[100] == 1502501


KeyError: 100

In [ ]:
# 3f) Explain this code excerpt 
# Note: you do not have to use this line of code at this point in the assignmnet.
#  It is one of the lines provided to you in 2e. Here, just write a quick comment on what it does. 

# In the cell below, explain in words what what the following line of code is doing:
population = list(df_zip.loc[df_zip['zip'] == zip_code]['population'])

YOUR ANSWER HERE

In [ ]:
# 3g) Masking the Zip Codes 

# In this part, you should write a for loop, updating the df_users dataframe.
# Go through each user, and update their zip-code, to Safe Harbour specifications:
#   If the user is from a zip code for the which the
#     "Geographic Subdivision" is less than equal to 20000:
#        - Change the zip code to 0 
#   Otherwise:
#         - Change the zip code to be only the first 3 numbers of the full zip cide
# Do all this re-writting the zip_code columns of the 'df_users' DataFrame
#
# Hints:
#  - This will be several lines of code, looping through the DataFrame, 
#      getting each zip code, checking the geographic subdivision with 
#      the population in zip_dict, and settig the zip_code accordingly. 

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert len(df_users) == 993
assert sum(df_users.zip == 0) == 7
assert df_users.loc[671, 'zip'] == 359


In [ ]:
# 3h) Save out the properly anonymized data to json file 

# Save out df_users as a json file, called 'real_anon_user_dat.json'

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert isinstance(pd.read_json('real_anon_user_dat.json'), pd.DataFrame)

Congrats, you're done! The users identities are much more protected now. 

Submit this notebook file to TritonED.